In [1]:
# to read and wrangle data
import pandas as pd

# to import data from LA Data portal
from sodapy import Socrata

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
housing_survey = pd.read_csv(r'data\GeoJson\tab\CHS2021ECL_PUMF.csv')

In [3]:
housing_survey.shape

(40988, 109)

In [4]:
housing_survey.columns

Index(['PUMFID', 'EHA_10', 'EHA_10A', 'EHA_10B', 'EHA_25', 'DWS_05A',
       'DWI_05A', 'DWI_05B', 'DWI_05C', 'DWI_05D',
       ...
       'PRSPIMST', 'PSCR_05', 'PSCR_10', 'PSCR_25', 'PSCR_35', 'PSCR_D40',
       'PSTIR_GR', 'PVISMIN', 'PWSA_D15', 'VERDATE'],
      dtype='object', length=109)

In [5]:
housing_survey['PGEOGR'].values

array([ 3, 26, 22, ..., 24,  5, 24], dtype=int64)

In [15]:
sub = housing_survey[housing_survey['PGEOGR']==23]

In [19]:
sub['PGEOGR'].unique()

array([23], dtype=int64)

In [24]:
pd.set_option('display.max_columns', None)
sub.head(20)


,PUMFID,EHA_10,EHA_10A,EHA_10B,EHA_25,DWS_05A,DWI_05A,DWI_05B,DWI_05C,DWI_05D,NES_05A,NSC_30A,NSC_30B,NSC_30C,NEI_05A,NEI_05B,NEI_05C,NEI_05D,NEI_05E,NEI_05F,NEI_05G,NEI_05H,NEI_05I,WSA_05,SDH_05,CER_05,CER_20,LIS_10,COS_10,COS_15,GH_05,GH_10,REGION,PAGEGR1,PAGEGR2,PAGEGR3,PAGEGR4,PAGEP1,PCER_10,PCER_15,PCHN,PCOS_05,PDCLASS,PDCT_05,PDCT_20,PDCT_25,PDTYPER,PDV_SAH,PDV_SHCO,PDV_SUIT,PDWLTYPE,PDWS_05,PDWS_10A,PDWS_10B,PDWS_10C,PDWS_10D,PDWS_10E,PDWS_10F,PDWS_10G,PDWS_10H,PDWS_10I,PDWS_10J,PEHA_05A,PEHA_05B,PEHA_05C,PEMPL,PFTHB5YR,PFWEIGHT,PGEOGR,PHGEDUC,PHHSIZE,PHHTTINC,PHTYPE,PLIS_05,PNES_05,PNSC_15,POWN_20,POWN_80,PPAC_05,PPAC_10,PPAC_23,PPAC_30,PPAC_35,PPAC_45A,PPAC_45C,PPAC_45D,PPAC_45E,PPAC_45F,PPAC_45G,PPAC_45H,PPAC_45I,PPAC_45J,PPAC_45K,PPAC_45L,PPAC_45M,PPAC_45N,PPAC_45O,PPROV,PRSPGNDR,PRSPIMST,PSCR_05,PSCR_10,PSCR_25,PSCR_35,PSCR_D40,PSTIR_GR,PVISMIN,PWSA_D15,VERDATE
81,63582,3,6,6,2,2,2,2,2,2,2,3,1,3,2,4,4,2,4,3,3,4,2,2,2,2,2,2,3,2,3,3,4,2,2,1,2,2,96,6,2,7,0,1,2,2,0,6,500.00,1,2,2,4,2,1,1,3,2,2,4,4,4,2,2,2,1,2,885.3046,23,7,2,150000,2,5,2,2,2,99999996,4,2,2,1,6,2,2,2,2,2,2,2,2,2,2,2,2,2,1,48,1,1,6,6,96,6,999.6,1,2,999.6,30/11/2022
100,63601,2,6,1,1,1,2,2,2,2,1,2,1,2,4,4,4,4,4,4,4,4,4,2,2,2,3,2,3,2,3,2,4,2,2,1,2,3,96,6,2,7,0,2,2,1,0,2,1600.00,1,6,3,2,2,3,2,2,2,1,2,2,2,1,2,2,1,6,672.1622,23,7,2,52500,2,5,3,2,6,99999996,4,2,1,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,48,1,2,2,2,3,6,999.6,2,1,999.6,30/11/2022
163,63664,1,1,6,1,3,2,2,2,2,3,1,1,1,4,4,4,4,4,4,4,4,4,2,2,1,3,2,2,2,5,4,4,2,2,1,2,3,2,1,2,4,2,2,2,1,3,2,1600.00,1,6,3,1,1,4,1,2,1,1,3,4,4,1,1,1,2,6,262.1698,23,4,1,60000,5,1,3,1,6,99999996,2,1,6,1,6,2,2,2,2,2,1,2,2,2,2,2,2,2,2,48,1,1,2,2,3,6,999.6,2,2,999.6,30/11/2022
283,63784,2,6,1,2,3,2,1,2,2,2,3,2,3,4,4,2,4,4,2,4,4,4,2,1,2,3,2,9,4,1,1,4,2,2,1,2,3,96,6,2,1,1,1,3,1,9,9,9999999.99,1,1,1,2,2,2,2,2,3,2,4,2,2,2,2,2,1,2,552.9252,23,7,2,125000,3,1,1,2,1,250000,4,1,1,2,1,2,2,2,2,2,1,2,2,2,2,2,1,2,2,48,2,2,9,9,99,6,999.6,1,9,999.6,30/11/2022
342,63843,1,2,6,1,3,2,2,2,2,3,1,1,1,4,4,4,4,4,4,4,4,4,2,2,2,3,2,3,2,5,5,4,2,2,1,2,3,96,6,1,3,2,1,4,1,3,6,2100.00,1,1,3,1,1,2,2,1,4,1,1,1,1,1,2,1,1,2,455.2997,23,4,1,30000,5,2,3,2,1,99999999,3,2,6,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,48,2,1,6,6,96,6,999.6,3,2,999.6,30/11/2022
429,63930,3,6,6,2,3,2,2,2,2,3,3,2,3,2,3,3,4,4,4,4,4,3,2,2,2,3,1,2,4,2,2,4,9,9,9,9,2,96,6,2,2,0,1,2,2,0,6,2200.00,1,6,1,2,2,2,3,4,3,2,2,2,2,2,2,2,1,2,647.5503,23,99,99,95000,5,6,2,2,1,70000,4,2,9,3,6,2,2,2,1,2,2,2,2,2,2,2,2,2,2,48,1,1,6,6,96,6,999.6,1,2,999.6,30/11/2022
451,63952,4,6,6,2,3,2,2,2,2,3,2,1,3,4,3,4,3,4,4,4,4,4,2,2,2,3,3,3,4,2,3,4,9,9,9,9,3,96,6,2,7,0,1,2,1,0,6,9999999.99,1,6,2,1,1,1,1,1,1,1,2,1,1,2,2,2,9,2,423.8816,23,7,99,99999999999,5,7,3,4,1,170000,3,2,9,1,6,2,2,2,2,2,1,2,2,2,2,2,2,2,2,48,2,1,6,6,96,6,999.6,1,2,999.6,30/11/2022
458,63959,2,6,1,2,3,2,2,2,2,3,2,1,2,4,4,4,3,4,4,4,4,4,2,2,1,3,3,3,2,2,1,4,1,2,1,2,3,1,4,2,8,0,1,4,2,0,6,3300.00,1,1,3,1,1,3,2,2,3,1,3,2,2,2,2,2,1,2,831.4841,23,6,5,165000,1,8,3,1,1,160000,3,1,1,1,6,2,2,2,2,2,2,2,1,2,2,2,2,2,2,48,2,1,6,6,96,6,999.6,1,2,999.6,30/11/2022
501,64002,2,6,1,2,3,2,2,2,2,3,1,3,1,4,3,3,3,3,3,3,3,3,2,2,2,3,1,1,2,1,3,4,2,2,1,2,2,96,6,2,2,0,2,1,1,0,2,1400.00,1,3,3,2,1,3,1,1,1,1,1,1,1,1,2,2,1,6,812.3267,23,3,1,42000,5,6,3,2,6,99999996,1,1,6,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,48,2,1,2,2,3,6,999.6,2,1,999.6,30/11/2022
504,64005,2,6,2,2,3,2,1,2,2,2,3,1,3,3,1,1,2,3,1,1,4,4,2,6,2,3,2,3,4,4,4,4,9,9,9,9,2,96,6,1,4,9,2,1,2,9,1,500.00,2,6,1,4,3,2,4,4,3,4,3,2,3,1,2,1,9,6,116.1544,23,99,3,25000,1,2,1,3,6,99999996,2,1,1,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,48,1,9,2,1,1,2,999.6,1,9,999.6,30/11/2022
